In [115]:
# se importan las librerias a utilizar en el proceso de ETL
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

In [217]:
# Se carga en la variable df_movies utilizando pandas, la informacion contenida en el archivo movies_dataset.csv
df_movies = pd.read_csv('./Dataset/movies_dataset.csv',delimiter=',')

# Muestro la primera fila del dataframe df_movies
df_movies.head(1)

C:\Users\Guille\AppData\Local\Temp\ipykernel_22616\3884198327.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('./Dataset/movies_dataset.csv',delimiter=',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [194]:
# Utilizo la funcion isnull() para detectar valores vacios y luego la funcion sum() para sumar la cantidad de valores vacios segun la columna.
df_movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [195]:
# Consulto el tipo de dato de cada columna.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

CAMBIOS OBLIGATORIOS

In [218]:
# Se crea una variable del tipo lista llamada 'columns' en la cual se carga los nombres de las columnas a tratar en el siguente codigo.
columns = ['revenue','budget']
# se utiliza la funcion fillna() para reemplazar los valores nulos por el numero entero 0.
df_movies[columns] = df_movies[columns].fillna(0)

#df_movies['release_date'] = df_movies['release_date'].fillna('no_release_date')
df_movies.dropna(subset=['release_date'], inplace=True)

enteros = ['1','12','22']
for a in enteros:
    df_movies = df_movies.drop(df_movies[df_movies['release_date'] == a].index)

df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

df_movies['release_year'] = df_movies['release_date'].dt.year

list_convert = ['revenue','budget']
df_movies[list_convert] = df_movies[list_convert].astype(float)

df_movies['return'] = np.where((df_movies['revenue'] != 0) & (df_movies['budget'] != 0), df_movies['revenue'] / df_movies['budget'], 0)

df_movies.drop(['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1,inplace=True)

***

CONVERSION DE TIPO DE VARIABLES

In [219]:
df_movies[['popularity']] = df_movies[['popularity']].astype(float)
df_movies['id'] = df_movies['id'].astype(int)

In [220]:

# creo un ciclo for que conprueba si los primeros valores de cada columna son del tipo string
columnas_texto = []
for columna in df_movies.columns:
    if isinstance(df_movies[columna][0], str):
        columnas_texto.append(columna)

# Muestra la lista con los nombres de las columnas que contienen 
print(columnas_texto)

# Analizo las columnas y sis valores
df_movies[columnas_texto].head(2)


['belongs_to_collection', 'genres', 'original_language', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'status', 'title']


,belongs_to_collection,genres,original_language,overview,production_companies,production_countries,spoken_languages,status,title
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji


Se observan en cada valor de algunas columnas que ahi esctructuras de listas de diccioonarios pero almacenadas como texto
Se procede a convertir los valores al tipo de dato correcto para por utilizar de mejor forma la informacion

In [221]:
lista_columnas_texto = ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages']

In [222]:
# Funcion que utiliza a su vez la funcion eval()
def convert_to_dicc(text):
    return eval(text)

for a in lista_columnas_texto:
    df_movies[a] = df_movies[a].fillna('None')
    df_movies[a] = df_movies[a].apply(convert_to_dicc)

In [223]:
# creo un ciclo for que comprueba si los primeros valores de cada columna son del tipo list
columnas = []
for columna in df_movies.columns:
    primer_valor = df_movies[columna][0]
    if type(primer_valor) == list or type(primer_valor) == dict:
        columnas.append(columna)

# Muestra la lista con los nombres de las columnas que contienen 
print(columnas)
df_movies[columnas].head(2)

['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']


,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
1,None,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."


In [224]:
def extract_name(list_dict):
    if type(list_dict) == list and len(list_dict) > 0:
        list_temp = []
        for dicc in list_dict:
            list_temp.append(dicc['name'])
        return list_temp
    if type(list_dict) == dict:
        return list_dict['name']
    else:
        return None
    
for column in lista_columnas_texto:
    df_movies[column] = df_movies[column].apply(extract_name)

df_movies[lista_columnas_texto].head()

,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,Toy Story Collection,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[English]
1,None,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]"
2,Grumpy Old Men Collection,"[Romance, Comedy]","[Warner Bros., Lancaster Gate]",[United States of America],[English]
3,None,"[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[United States of America],[English]
4,Father of the Bride Collection,[Comedy],"[Sandollar Productions, Touchstone Pictures]",[United States of America],[English]


In [225]:
# Establesco la columna id como indice antes cambiandole el nombre a 'IdMovie'
columna_id = df_movies['id']
df_movies.drop('id',inplace=True,axis=1)
df_movies.insert(0, 'IdMovie', columna_id)
df_movies = df_movies.sort_values('IdMovie')
df_movies = df_movies.set_index('IdMovie')

In [205]:
df_credits = pd.read_csv('./Dataset/credits.csv',delimiter=',')
df_credits.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [119]:
# creo un ciclo for que conprueba si los primeros valores de cada columna son del tipo string
columnas_texto = []
for columna in df_credits.columns:
    primer_valor = df_credits[columna][0]
    if isinstance(primer_valor, str):
        columnas_texto.append(columna)

# Muestra la lista con los nombres de las columnas que contienen 
print(columnas_texto)

['cast', 'crew']


In [121]:
# Funcion que utiliza a su vez la funcion eval()
def convert_to_dicc(text):
    return eval(text)

for a in columnas_texto:
    df_credits[a] = df_credits[a].fillna('None')
    df_credits[a] = df_credits[a].apply(convert_to_dicc)

TypeError: eval() arg 1 must be a string, bytes or code object

In [123]:
def reductor_dicc(lista):
    if type(lista) == list and len(lista) > 0:
        keys = list(lista[0].keys())
        dicc_temp = {}
        for key in keys:
            dicc_temp[key] = []
    
        for dicc in lista:
            for key in keys:
                dicc_temp[key].append(dicc[key])
        return dicc_temp
    else:
        return 0

In [124]:
for a in ['cast','crew']:
    df_credits[a] = df_credits[a].apply(reductor_dicc)
    

In [125]:
df_credits

,cast,crew,id
0,"{'cast_id': [14, 15, 16, 17, 18, 19, 20, 26, 2...","{'credit_id': ['52fe4284c3a36847f8024f49', '52...",862
1,"{'cast_id': [1, 8, 2, 24, 10, 25, 26, 11, 14, ...","{'credit_id': ['52fe44bfc3a36847f80a7cd1', '52...",8844
2,"{'cast_id': [2, 3, 4, 5, 6, 9, 10], 'character...","{'credit_id': ['52fe466a9251416c75077a89', '52...",15602
3,"{'cast_id': [1, 2, 3, 4, 5, 6, 8, 10, 20, 21],...","{'credit_id': ['52fe44779251416c91011acb', '52...",31357
4,"{'cast_id': [1, 2, 3, 4, 13, 14, 15, 16, 17, 1...","{'credit_id': ['52fe44959251416c75039ed7', '52...",11862
...,...,...,...
45471,"{'cast_id': [0, 1, 2], 'character': ['', '', '...","{'credit_id': ['5894a97d925141426c00818c', '58...",439050
45472,"{'cast_id': [1002, 1008, 1007, 1005, 1015, 100...","{'credit_id': ['52fe4af1c3a36847f81e9b15', '52...",111109
45473,"{'cast_id': [6, 7, 8, 10, 11, 12, 13, 9, 14, 1...","{'credit_id': ['52fe4776c3a368484e0c8387', '52...",67758
45474,"{'cast_id': [2, 3, 4, 5, 6], 'character': ['',...","{'credit_id': ['533bccebc3a36844cf0011a7', '58...",227506


In [43]:
list_df = []
for ld in df_credits['cast']:
    if type(ld) == dict:
        df_temp = pd.DataFrame(ld)
        list_df.append(df_temp)
        
df_cast = pd.concat(list_df, ignore_index=True)

In [44]:
list_df = []
for ld in df_credits['crew']:
    if type(ld) == dict:
        df_temp = pd.DataFrame(ld)
        list_df.append(df_temp)
        
df_crew = pd.concat(list_df, ignore_index=True)

***

AJUSTE DE LA BASE DE DATOS PARA LA API

In [91]:
df_movies.drop(['belongs_to_collection','genres','original_language','overview','production_companies',
                'production_countries','runtime','spoken_languages','tagline',],axis=1,inplace=True)

In [92]:
# Crear nueva columna con el día de la semana en formato de texto
df_movies['day'] = df_movies['release_date'].dt.strftime("%A")

In [138]:
df_movies['release_month'] = df_movies['release_date'].dt.month

In [126]:
# Establesco la columna id como indice antes cambiandole el nombre a 'IdMovie'
columna_id = df_credits['id']
df_credits.drop('id',inplace=True,axis=1)
df_credits.insert(0, 'IdMovie', columna_id)
df_credits = df_credits.sort_values('IdMovie')
df_credits = df_credits.set_index('IdMovie')

In [127]:
df_movies = pd.merge(df_movies, df_credits, on='IdMovie')

In [128]:
def name_actor(dicc):
    if type(dicc) == dict: 
        #print(dicc['name'])
        lst = dicc['name']
        return lst
df_movies['cast'] = df_movies['cast'].apply(name_actor)

In [130]:
def name_actor(dicc):
    if type(dicc) == dict:
        lst_temp = []
        for index,job in enumerate(dicc['job']):
            if job == 'Director':
                lst_temp.append(dicc['name'][index])
        return lst_temp
    else:
        return 'no director'
df_movies['crew'] = df_movies['crew'].apply(name_actor)

In [135]:
df_movies.rename(columns={'cast': 'actor','crew':'director'},inplace=True)

In [139]:
df_movies.head(2)

,budget,popularity,release_date,revenue,status,title,vote_average,vote_count,release_year,return,day,actor,director,release_month
IdMovie,,,,,,,,,,,,,,
2,0.0,3.860491,1988-10-21,0.0,Released,Ariel,7.1,44.0,1988,0.0,Friday,"[Turo Pajala, Susanna Haavisto, Matti Pellonpä...",[Aki Kaurismäki],10
3,0.0,2.292110,1986-10-16,0.0,Released,Shadows in Paradise,7.1,35.0,1986,0.0,Thursday,"[Matti Pellonpää, Kati Outinen, Sakari Kuosman...",[Aki Kaurismäki],10


In [117]:
df_movies.drop(['cast','crew'],inplace=True,axis=1)

In [140]:
df_movies.to_csv('api.csv', index=False)

In [136]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44936 entries, 2 to 469172
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4466 non-null   object        
 1   budget                 44936 non-null  float64       
 2   genres                 42609 non-null  object        
 3   original_language      44926 non-null  object        
 4   overview               44029 non-null  object        
 5   popularity             44936 non-null  float64       
 6   production_companies   33378 non-null  object        
 7   production_countries   38855 non-null  object        
 8   release_date           44936 non-null  datetime64[ns]
 9   revenue                44936 non-null  float64       
 10  runtime                44696 non-null  float64       
 11  spoken_languages       41240 non-null  object        
 12  tagline                20287 non-null  object        
 13  

***

MACHINE LEARNING

In [226]:
df_movies.head(2)

,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
IdMovie,,,,,,,,,,,,,,,,,,,
2,None,0.0,"[Drama, Crime]",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[Villealfa Filmproduction Oy, Finnish Film Fou...",[Finland],1988-10-21,0.0,69.0,"[suomi, Deutsch]",Released,NaN,Ariel,7.1,44.0,1988,0.0
3,None,0.0,"[Drama, Comedy]",fi,"An episode in the life of Nikander, a garbage ...",2.292110,[Villealfa Filmproduction Oy],[Finland],1986-10-16,0.0,76.0,"[English, suomi, svenska]",Released,NaN,Shadows in Paradise,7.1,35.0,1986,0.0


In [227]:
# Se eliminan las filas donde el valor status no sea igual a released/estrenado ya que segun la consigna se debe trabajar con las peliculas estrenadas
df_movies = df_movies[df_movies['status'] == 'Released']

In [228]:
# Elimino la columna status ya que todas las peliculas tiene el status de estrenadas
df_movies.drop('status',inplace=True,axis=1)

In [230]:
# Elimino columnas que no aportan informacion relevante para la etapa de ML
drops = ['original_language','belongs_to_collection','tagline','production_companies','production_countries','release_date','runtime','spoken_languages']
df_movies.drop(drops,inplace=True,axis=1)

In [237]:
df_movies.head(5)

,budget,genres,overview,popularity,revenue,title,vote_average,vote_count,release_year,return
IdMovie,,,,,,,,,,
2,0.0,"[Drama, Crime]",Taisto Kasurinen is a Finnish coal miner whose...,3.860491,0.0,Ariel,7.1,44.0,1988,0.000000
3,0.0,"[Drama, Comedy]","An episode in the life of Nikander, a garbage ...",2.292110,0.0,Shadows in Paradise,7.1,35.0,1986,0.000000
5,4000000.0,"[Crime, Comedy]",It's Ted the Bellhop's first night on the job....,9.026586,4300000.0,Four Rooms,6.5,539.0,1995,1.075000
6,0.0,"[Action, Thriller, Crime]","While racing to a boxing match, Frank, Mike, J...",5.538671,12136938.0,Judgment Night,6.4,79.0,1993,0.000000
11,11000000.0,"[Adventure, Action, Science Fiction]",Princess Leia is captured and held hostage by ...,42.149697,775398007.0,Star Wars,8.1,6778.0,1977,70.490728


In [257]:
df_movies.isna().sum()

budget             0
overview           0
popularity         0
revenue            0
title              0
vote_average       0
vote_count         0
release_year       0
return             0
Drama              0
Crime              0
Comedy             0
Action             0
Thriller           0
Adventure          0
Science Fiction    0
Animation          0
Family             0
Romance            0
Mystery            0
Music              0
Horror             0
Fantasy            0
Documentary        0
War                0
Western            0
History            0
Foreign            0
TV Movie           0
dtype: int64

In [234]:
df_movies.dropna(subset=['genres','overview'], inplace=True)

In [235]:
unicos = []
for a in df_movies['genres']:
    if type(a) == list:
        for i in a:
            if i not in unicos:
                unicos.append(i)
print(unicos)

['Drama', 'Crime', 'Comedy', 'Action', 'Thriller', 'Adventure', 'Science Fiction', 'Animation', 'Family', 'Romance', 'Mystery', 'Music', 'Horror', 'Fantasy', 'Documentary', 'War', 'Western', 'History', 'Foreign', 'TV Movie']


In [238]:
# Crear columnas nuevas en el DataFrame para cada género único y asignar valores 1 o 0
for genre in unicos:
    df_movies[genre] = df_movies['genres'].apply(lambda x: 1 if genre in x else 0)

In [239]:
df_movies.drop('genres',inplace=True,axis=1)

In [260]:
df_movies

,title,overview
IdMovie,,
2,Ariel,Taisto Kasurinen is a Finnish coal miner whose...
3,Shadows in Paradise,"An episode in the life of Nikander, a garbage ..."
5,Four Rooms,It's Ted the Bellhop's first night on the job....
6,Judgment Night,"While racing to a boxing match, Frank, Mike, J..."
11,Star Wars,Princess Leia is captured and held hostage by ...
...,...,...
464819,The Making of 'The Terminator': A Retrospective,An interview session with Arnold Schwarzenegge...
465044,Abduction,A horror comedy spoofing conspiracy theory mov...
467731,Tragedy in a Temporary Town,Fifteen-year-old girl Dotty Fisher is assaulte...


In [242]:
df_movies.to_csv('ML.csv')

In [259]:
df_movies = df_movies[['title','overview']]

In [264]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [307]:
model = TfidfVectorizer(max_df = 0.9)

In [308]:
matrix = model.fit_transform(df_movies["overview"])

In [309]:
index = NearestNeighbors(n_neighbors=6, algorithm='kd_tree')
index.fit(matrix)

NearestNeighbors(algorithm='kd_tree', n_neighbors=6)

In [310]:
def recommend_movies(title):
    overview = df_movies[df_movies['title'] == title]['overview'].iloc[0] # Busca el valor de la columna 'overview' segun el titulo ingresado
    overview_vector = model.transform([overview])  # Transforma el título de entrada en una representación vectorial
    _, indices = index.kneighbors(overview_vector)  # Busca los vecinos más cercanos basados en la representación vectorial
    recommendations = df_movies["title"].iloc[indices[0][1:]]  # Obtiene los títulos de las películas más similares (excluyendo la película de referencia)
    return recommendations

In [311]:
movie_title = "Manuel on the Island of Wonders"  # Ingresa el título de la película de referencia
recommendations = recommend_movies(movie_title)
print("Recomendaciones de películas:")
for movie in recommendations:
    print(movie)

Recomendaciones de películas:
Slaves of New York
Game Over
Afstiros katallilo
Wojaczek
10,000 Dollars for a Massacre
